In [1]:
import nibabel as nib
from confounds import censor_method
import numpy as np
import os

In [3]:
data = "/scratch/jjeyachandra/niviz/data"
sub = "sub-LA1LA20002"
space = "MNINonLinear"
entities = "ses-01_task-rest_run-1_bold"

dtseries = nib.load(
    os.path.join(data,sub,space,'Results',
    entities,f"{entities}_Atlas.dtseries.nii")
)

`dataobj` contains the array data that is formatted

In [7]:
dtseries.dataobj.shape

(106, 91282)

Exploring the `cifti2_axes.Axis` object

Matches connectome workbench:
- `axis=0` refers to the time-series rows
- `axis=1` refers to the brain model columns

In [24]:
series_ax = dtseries.header.get_axis(0)
brain_ax = dtseries.header.get_axis(1)

In a `SeriesAxis` the following properties are defined:
- `start` - start TR
- `step` - TR 
- `size` - number of samples
- `unit` - unit of samples

This is mainly used for type checking, it doesn't hold any actual data!

In [23]:
series_ax.start, series_ax.step, series_ax.size, series_ax.unit

(0.0, 0.8, 106, 'SECOND')

In a `BrainModelAxis` axis we have:

- `name` - which is an array with vertex --> structure mappings
- `voxel` - (N,3) array containing voxel indices (-1,-1,-1) is used to indicate a surface point
- `vertex` - (N,) array with similar function to `voxel` but indicating vertices instead
- `affine` - Affine transformation mapping voxel indices
- `volume_shape` - shape of volume component
- `nvertices` - `{STRUCTURE: NVERTEX}` mapping

In [28]:
print(brain_ax.name, brain_ax.name.shape)


['CIFTI_STRUCTURE_CORTEX_LEFT' 'CIFTI_STRUCTURE_CORTEX_LEFT'
 'CIFTI_STRUCTURE_CORTEX_LEFT' ... 'CIFTI_STRUCTURE_THALAMUS_RIGHT'
 'CIFTI_STRUCTURE_THALAMUS_RIGHT' 'CIFTI_STRUCTURE_THALAMUS_RIGHT'] (91282,)


In [37]:
brain_ax.nvertices

{'CIFTI_STRUCTURE_CORTEX_LEFT': 32492, 'CIFTI_STRUCTURE_CORTEX_RIGHT': 32492}

`BrainModelAxis` methods:

- `iter_structures`: (BRAINSTRUCT, slice, brain model covering specific region)
- `to_mapping`: Convert into a `MatrixIndicesMap` for storage in CIFTI-2 format

In [40]:
list(brain_ax.iter_structures())[:4]

[('CIFTI_STRUCTURE_CORTEX_LEFT',
  slice(0, 29696, None),
  <nibabel.cifti2.cifti2_axes.BrainModelAxis at 0x7f74066e6310>),
 ('CIFTI_STRUCTURE_CORTEX_RIGHT',
  slice(29696, 59412, None),
  <nibabel.cifti2.cifti2_axes.BrainModelAxis at 0x7f74062d4640>),
 ('CIFTI_STRUCTURE_ACCUMBENS_LEFT',
  slice(59412, 59547, None),
  <nibabel.cifti2.cifti2_axes.BrainModelAxis at 0x7f74062d4d60>),
 ('CIFTI_STRUCTURE_ACCUMBENS_RIGHT',
  slice(59547, 59687, None),
  <nibabel.cifti2.cifti2_axes.BrainModelAxis at 0x7f74066e4040>)]

`cifti2.Cifti2MatrixIndicesMap`

Provides a mapping between matrix indices and their interpretation. The `MatrixIndicesMap` object can be used for all types of CIFTI-2 objects. The attributes stored are dependent on the data-type itself (i.e BrainModel, NamedMap, Parcel, Surface etc..)

`BrainModel` variant

In [80]:
mapping = brain_ax.to_mapping(0)
mapping.indices_map_to_data_type

'CIFTI_INDEX_TYPE_BRAIN_MODELS'

In [81]:
brain_models = list(mapping.brain_models)
brain_models

In [82]:
print([b.model_type for b in brain_models])

['CIFTI_MODEL_TYPE_SURFACE', 'CIFTI_MODEL_TYPE_SURFACE', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS', 'CIFTI_MODEL_TYPE_VOXELS']


In [76]:
print([b.brain_structure for b in brain_models])

['CIFTI_STRUCTURE_CORTEX_LEFT', 'CIFTI_STRUCTURE_CORTEX_RIGHT', 'CIFTI_STRUCTURE_ACCUMBENS_LEFT', 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT', 'CIFTI_STRUCTURE_AMYGDALA_LEFT', 'CIFTI_STRUCTURE_AMYGDALA_RIGHT', 'CIFTI_STRUCTURE_BRAIN_STEM', 'CIFTI_STRUCTURE_CAUDATE_LEFT', 'CIFTI_STRUCTURE_CAUDATE_RIGHT', 'CIFTI_STRUCTURE_CEREBELLUM_LEFT', 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT', 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT', 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT', 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT', 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT', 'CIFTI_STRUCTURE_PALLIDUM_LEFT', 'CIFTI_STRUCTURE_PALLIDUM_RIGHT', 'CIFTI_STRUCTURE_PUTAMEN_LEFT', 'CIFTI_STRUCTURE_PUTAMEN_RIGHT', 'CIFTI_STRUCTURE_THALAMUS_LEFT', 'CIFTI_STRUCTURE_THALAMUS_RIGHT']


`Series` variant, only contains basic information about SeriesAxis object

In [89]:
# The dim argument doesn't seem to make a difference
mapping = series_ax.to_mapping(1)
mapping.indices_map_to_data_type

'CIFTI_INDEX_TYPE_SERIES'

In [90]:
mapping.series_start, mapping.series_step, mapping.series_unit

(0.0, 0.8, 'SECOND')

In [92]:
mapping.number_of_series_points

106

Constructing a new valid `dtseries` object:

1. We can leave the BrainModelAxis be
2. We have to construct a new SeriesAxis

In [119]:
DROP_TR = 5
series_ax = dtseries.header.get_axis(0)
data = np.asanyarray(dtseries.dataobj)
nifti_header = dtseries.nifti_header

Let's drop a few frames from `data`

In [134]:
data_dropped = data[DROP_TR:, :].copy()
data_dropped.shape

(101, 91282)

Now let's update the old `SeriesAxis`

In [117]:
from nibabel.cifti2 import cifti2_axes
from nibabel.cifti2 import Cifti2Image

In [115]:
new_series_ax = cifti2_axes.SeriesAxis(
    start=series_ax.start + DROP_TR*series_ax.step,
    step = series_ax.step,
    size = series_ax.size - DROP_TR,
    unit = series_ax.unit
)

Now we can construct a new header object

In [116]:
new_header = cifti2_axes.to_header([
    new_series_ax, brain_ax
])

Finally we can associate the dataobj with the header

In [144]:
dtseries.get_fdata().shape

(106, 91282)

In [147]:
new_dtseries = Cifti2Image(
    dataobj=data_dropped,
    header=new_header
)

nib.save(new_dtseries,'test.dtseries.nii')

You can now verify in `connectome-workbench` whether the result is as expected